Review well Fitted  spectra
----------------------------------

First let's import the packages we will need.


Analyse outputs from GELATO fit results on a signe spectrum

- author : Sylvie Dagoret-Campagne
- creation date : 2024-05-25
- update : 2024-05-25


- Kernel at CCIN2P3 : ``conda_desc_py310_pcigale``
- Kernel on my laptop : ``pcigale``

In [ ]:
# Import packages
import gelato
import numpy as np
%matplotlib inline
import matplotlib as mpl
mpl.rcParams['font.size'] = 25
from matplotlib import pyplot # For plotting
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
# For loading in data
from astropy.io import fits
from astropy.table import Table 
import os,re
import pandas as pd

In [ ]:
from astropy.modeling import models, fitting
from astropy import modeling
# define a model for a line
g_init = models.Gaussian1D(amplitude=1, mean=0, stddev=1)
# initialize a linear fitter
fit_g = fitting.LevMarLSQFitter()

In [ ]:
from fors2pcigale.fors2starlightio import Fors2DataAcess

In [ ]:
#from gelato.Plotting import  Plot, PlotFig,subplotplot
#from gelato.Plotting import subplotplot
from gelato.Plotting import logbarrier
from scipy.optimize import minimize

#import gelato.ConstructParams as CP

import gelato.Utility as U
import gelato.Plotting as P
import gelato.ConstructParams as CP

# GELATO
import gelato.Utility as U
import gelato.CustomModels as CM
import gelato.SpectrumClass as SC

from gelato.Constants import C

In [ ]:
#from libExampleFitInNb import *

## Fors2 Interface

In [ ]:
fors2 = Fors2DataAcess()

## Config

In [ ]:
version = "v3"

## Gelato Parameters

In [ ]:
# Path to the parameters file
#path_params = './ExampleParametersFitInNb.json'
path_params = f"./ExampleParametersFitInNb_{version}.json"

# Create Parameters dictionary
params_gel = gelato.ConstructParams.construct(path_params)

# Set to not multiprocessing
params_gel['NProcess'] = 1
params_gel['Verbose'] = False

In [ ]:
params_gel['EmissionGroups']

In [ ]:
for group in params_gel['EmissionGroups']:
    info_group = "Group : "+ group["Name"]
    print(info_group)
    all_species = group['Species']
    for the_species in all_species:
        #info_species = "\t  Species : " + the_species['Name'] + " FlagGroup :: " +  the_species['FlagGroups'][0] + "Nlines = " + str(len( the_species['Lines'] )) 
        info_species = "\t  Species : " + the_species['Name']  + " , Nlines = " + str(len( the_species['Lines'] )) 
        print(info_species)

## Table with Spectra name and Redshifts

In [ ]:
#df_objlist = pd.read_csv("object_filelist_v0.csv",index_col=0)
filename_object_file_list = f"object_filelist_{version}.csv"
df_objlist = pd.read_csv(filename_object_file_list,index_col=0)

## Input files before the fit, sorting and index

In [ ]:
#path = "./spec_forgelato/v0"
path = f"./spec_forgelato/{version}"

In [ ]:
list_all_files = os.listdir(path)

In [ ]:
idx_selected_files = []
list_selected_files = []
for file in list_all_files:
    res = re.findall("^specgelato_SPEC.*[.]fits$",file)
    if len(res):
        list_selected_files.append(file)
        num = int(re.findall("specgelato_SPEC(.*)[.]fits$",file)[0])   
        idx_selected_files.append(num)

In [ ]:
idx_selected_files = np.array(idx_selected_files)
list_selected_files = np.array(list_selected_files)
idx_sorted_files = np.argsort(idx_selected_files)
list_sorted_files = list_selected_files[idx_sorted_files]

In [ ]:
NSPEC = len(list_sorted_files)

In [ ]:
print(f"Number of Spectra : {NSPEC}")

### Output path

In [ ]:
#Output path 
output_path = params_gel['OutFolder']
print(f"output_path defined in json file : {output_path}")

In [ ]:
if not os.path.isdir(output_path):
    os.mkdir(output_path)

## Loop on files

In [ ]:
NStart = 0
NStop = 548
ListBadSpec = ["SPEC189","SPEC346"]

outputpathname  = os.path.basename(params_gel['OutFolder'])
filename_processing_final = f"runFinalStatus_fitgenato_{NStart}-{NStop}_{outputpathname}.csv"

if not os.path.exists(filename_processing_final):
    assert False

df_processing = pd.read_csv(filename_processing_final)
df_processing.set_index("tag", inplace = True)

# loop
for index in range(NStart,NStop+1):
        
    shortfilename = list_sorted_files[index]
    fullfilename = os.path.join(path,shortfilename) 
    path_spec = fullfilename
    tag_spec = re.findall(".*_(SPEC.*).fits$", shortfilename)

    #define outputfilenames and fullfilename
    output_filename = shortfilename.split('.')[0] + "-results.fits"
    output_filename_pulls = shortfilename.split('.')[0] + "-pulls-results.csv"
    output_filename_emissionlines = shortfilename.split('.')[0] + "-emissionlines-results.csv"

    output_path_fullfilename = os.path.join(output_path,output_filename)
    output_path_fullfilename_pulls = os.path.join(output_path,output_filename_pulls)
    output_path_fullfilename_emissionlines = os.path.join(output_path,output_filename_emissionlines)
    
    output_figfilename1 = shortfilename.split('.')[0] + "-spec.pdf"
    output_figfilename2 = shortfilename.split('.')[0] + "-fit.pdf"
    output_figfilename3 = shortfilename.split('.')[0] + "-comp.pdf"
    output_path_figfullfilename1 = os.path.join(output_path,output_figfilename1)
    output_path_figfullfilename2 = os.path.join(output_path,output_figfilename2)
    output_path_figfullfilename3 = os.path.join(output_path,output_figfilename3)
    
    #find the redshift
    if len(tag_spec)>0:
        tag_spec = tag_spec[0]
        all_inputspecfilenames = df_objlist.Path.values
        for idx_tag,filen in enumerate(all_inputspecfilenames):
            if tag_spec in filen:
                break

    df_row = df_objlist.iloc[idx_tag]
    redshift = df_row["z"]

   

    spec_name_sel = tag_spec
    title = f"{index}) {output_filename}, z={redshift:.3f}" 
    
    #print(index,spec_name_sel,"redshift",df_row,redshift)
    #print(title)

   

    # acess to the image array
    img = fors2.get_specimg(spec_name_sel)

    # get the image filename and path
    spec_sec_fileimg = fors2.get_specimgfile(spec_name_sel)


   

    

    # Show original image
    if 0:
        fig = plt.figure(constrained_layout=True,figsize=(12,6))
        plt.imshow(img)
        ax = plt.gca()
        # Hide X and Y axes label marks
        ax.xaxis.set_tick_params(labelbottom=False)
        ax.yaxis.set_tick_params(labelleft=False)
        # Hide X and Y axes tick marks
        ax.set_xticks([])
        ax.set_yticks([])
        plt.show()

    
    # case when the input file does not exist
    if not os.path.exists(path_spec) or spec_name_sel in ListBadSpec:
        msg = f"Input spectrum file {path_spec} does not exists or is wrong : SKIP"  
        print(msg)
        continue

    # test existence of files:
    # Existence in the processing 
    print("---------------------------------------------------------------")
    print(title)
    if spec_name_sel in df_processing.index:
        df_processing_row = df_processing.loc[spec_name_sel]
        print(df_processing_row)
    else:
        printf(f">>>>. tag {spec_name_sel} NOT Found in df_processing.index" )

    if os.path.exists(output_path_fullfilename):
        print(f"- {output_path_fullfilename} Exits ")
        
    else:
        print(f"- >>>>>> {output_path_fullfilename} DOES NOT Exits ")

     
    if os.path.exists(output_path_fullfilename_pulls):
        print(f"- {output_path_fullfilename_pulls} Exits ")
        df_pulls = pd.read_csv(output_path_fullfilename_pulls)
        print(df_pulls)
    else:
        print(f"- >>>>>> {output_path_fullfilename_pulls} DOES NOT Exits ")
        df_pulls = None

    if os.path.exists(output_path_fullfilename_emissionlines):
        print(f"- {output_path_fullfilename_emissionlines} Exits ")
        df_el = pd.read_csv(output_path_fullfilename_emissionlines)
        print(df_el)
    else:
        print(f"- >>>>>> {output_path_fullfilename_emissionlines} DOES NOT Exits ")
        df_el = None

    if os.path.exists(output_path_figfullfilename1):
        print(f"- {output_path_figfullfilename1} Exits ")
    else:
        print(f"- >>>>>> {output_path_figfullfilename1} DOES NOT Exits ")

    if os.path.exists(output_path_figfullfilename2):
        print(f"- {output_path_figfullfilename2} Exits ")
    else:
        print(f"- >>>>>> {output_path_figfullfilename2} DOES NOT Exits ")
        
    if os.path.exists(output_path_figfullfilename3):
        print(f"- {output_path_figfullfilename3} Exits ")
    else:
        print(f"- >>>>>> {output_path_figfullfilename3} DOES NOT Exits ")

    
    #Get the spectrum-result
    spectrum = Table.read(path_spec)
    # Start with inverse variance
    ivar = spectrum['ivar']
    good = ivar > 0 # GELATO only looks at points with nonzero weights
    # Finally, let's load in the data
    wavl = 10**spectrum['loglam'][good]
    flux = spectrum['flux'][good]
    ivar = ivar[good]
    args = (wavl,flux,ivar) # These will be useful later

 
    
    

# Save file status